Building likelihood maps for a set of representative images.

> TODO: use images from dataset and superimpose the bounding boxes on them

In [1]:
import retinoto_py as fovea
N_show = 6
args = fovea.Params(batch_size=N_show, do_mask=False, n_val_stop=N_show)
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	- Timestamp (UTC)  2025-12-03_16-04-14	 user laurentperrinet	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 2018 has been set.


Params(image_size=224, do_mask=False, do_fovea=False, rs_min=0.0, rs_max=-5.0, padding_mode='zeros', seed=2018, batch_size=6, num_workers=0, in_memory=True, model_name='convnext_base', do_scratch=False, num_epochs=20, n_train_stop=65536, n_val_stop=6, lr=3e-06, delta1=0.05, delta2=0.01, weight_decay=0.02, label_smoothing=0.03, shuffle=True, verbose=True)

In [ ]:
import torch
from torchvision.io import read_image

alpha = .5
s_max = 1000

type(full_image) = <class 'torch.Tensor'>, full_image.dtype = torch.float32, full_image.shape = torch.Size([3, 587, 800])


In [3]:
# VAL_DATA_DIR = args.DATAROOT / 'Imagenet_bbox' / 'val'

# args = fovea.Params(batch_size=N_show, do_mask=False, n_val_stop=N_show)
# val_dataset = fovea.get_dataset(args, VAL_DATA_DIR, n_stop=N_show)
# val_loader = fovea.get_loader(args, val_dataset, seed=seed)
# for images, labels in val_loader:
#     images, labels = images.to('cpu'), labels.to('cpu')
#     break
# fig, ax = fovea.imshow(images, fig_height=5)
# fovea.plt.show()


In [11]:
idx_to_label = fovea.get_idx_to_label(args)
label2idx = fovea.get_label_to_idx(args)
label2idx['impala']

Loading labels from local cache cached_data/imagenet_class_index.json...
Loading labels from local cache cached_data/imagenet_class_index.json...


352

In [14]:
for key in label2idx.keys(): 
    if 'owl' in key: print(key)

great_grey_owl
howler_monkey
mixing_bowl
soup_bowl


In [5]:
dataset = 'bbox'
model_filename = args.data_cache / f'33_fovea_model_name={args.model_name}_dataset={dataset}.pth'
model = fovea.load_model(args, model_filename=model_filename)
model_filename

loading .... cached_data/33_fovea_model_name=convnext_base_dataset=bbox.pth


PosixPath('cached_data/33_fovea_model_name=convnext_base_dataset=bbox.pth')

In [ ]:
for image_url, true_label in  [('./images/jaguar_5.jpg',  'jaguar'),
                                ('./images/frog.jpg',  'tree_frog'),
                                ('./images/Hidden_snow_leopard.png',  'snow_leopard'),
                                ('./images/Hidden_leopard.png',  'leopard'),
                                ('./images/Hidden_giraffe.png',  'giraffe'),
                                ('./images/Hidden_tiger.png',  'tiger'),
                                ('./images/Hidden_owl.png',  'great_grey_owl'),
                                ('./images/wolf.jpg',  'wolf'),
                                ]:
    full_image = read_image(image_url)[:3, :, :]/255
    full_image_np = torch.movedim(full_image, (1, 2, 0), (0, 1, 2)).numpy()
    print(f"{type(full_image) = }, {full_image.dtype = }, {full_image.shape = }")

    pos_H, pos_W, outputs = fovea.compute_likelihood_map(args, model, full_image, size_ratio=.6)
    outputs = outputs.cpu()
    logit_label = outputs[:, label2idx[true_label]]
    idx_max = logit_label.argmax()


    fig, ax = fovea.plt.subplots()
    ax.imshow(full_image_np)
    ax.scatter(pos_W.ravel(), pos_H.ravel(), s=logit_label.abs()*s_max, c=logit_label, alpha=alpha, edgecolors='none', cmap='coolwarm', vmin=-max(abs(logit_label)), vmax=max(abs(logit_label)),)

    ax.scatter(pos_W.ravel()[idx_max], pos_H.ravel()[idx_max], s=logit_label[idx_max]*s_max, marker='*', c='white', alpha=alpha)
    fig.set_facecolor(color='white')
    fovea.plt.show()

type(full_image) = <class 'torch.Tensor'>, full_image.dtype = torch.float32, full_image.shape = torch.Size([3, 587, 800])
